In [1]:
import pandas as pd
import numpy as np
import chardet
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)

with open("en.openfoodfacts.org.products.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [2]:
# chargement de la base et comptage des linges
#test pour Yosra
df = pd.read_csv('en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8', dtype=str)

In [3]:
#on met tout en majuscule
df = df.applymap(lambda s:s.lower() if type(s) == str else s)
df = df.drop(['code'], axis=1)
df.columns
print(f'Taille du df: {len(df.index)}')
print(f'Nombre de colonne: {len(df.columns)}')
df_columns = pd.DataFrame(df.columns)
df_columns.to_excel('column.xlsx')


Taille du df: 1768986
Nombre de colonne: 185


In [4]:
# On stock tous les nom de colonnes qui ne sont pas de URL
not_url_col = []
for i in df.columns:
   if 'url' not in i: 
    not_url_col.append(i)

In [5]:
# On selectionne les produits vendu et produit en France
df = df.replace(np.nan, '', regex=True)
df_FO = df[df['origins_tags'].str.contains("fr")]
df_FOS = df_FO[df_FO['origins_tags'].str.contains("fr")]
print(f'Taille du df: {len(df_FOS.index)}')
print(f'Nombre de colonne: {len(df_FOS.columns)}')
df = df.replace('', np.nan, regex=True)

Taille du df: 34803
Nombre de colonne: 185


In [6]:
# Premier nettoyage pour les colonnes qui ne sont pas des URL
df_FOS[not_url_col] = df_FOS[not_url_col].apply(lambda x: x.str.strip())
df_FOS[not_url_col]  = df_FOS[not_url_col] .replace('\s+', ' ', regex=True)
df_FOS[not_url_col]  = df_FOS[not_url_col] .replace('_', ' ', regex=True)
df_FOS[not_url_col]  = df_FOS[not_url_col] .replace(',', ', ', regex=True)

In [7]:
# Est-ce que c'est une bonne clé unique ?
# Les colonnes ci dessous sont sensé faire une clé unique
unique_col = ['product_name', 'brands', 'manufacturing_places', 'stores', 'generic_name', 'quantity', 'packaging', 'countries', 'origins']
print(f"{df_FOS[unique_col].duplicated().sum()} : clé unique")
print(f"{df_FOS.duplicated().sum()} : clé unique")
# non ce n'est pas une bonne clé unique

373 : clé unique
0 : clé unique


In [8]:
# on va drop les lignes qui ont le moins d'information
print(len(df_FOS))
df_FOS['count'] = df_FOS.isnull().sum(axis=1)
df_FOS= df_FOS.sort_values(['count'])
df_FOS = df_FOS.drop_duplicates(subset=unique_col,keep='first')
df_FOS = df_FOS.drop('count',1)
print(len(df_FOS))

34803
34430


In [9]:
# Est-ce que c'est une bonne clé unique ?
unique_col = ['product_name', 'brands', 'manufacturing_places', 'stores', 'generic_name', 'quantity', 'packaging', 'countries', 'origins']
print(f"{df_FOS[unique_col].duplicated().sum()} : clé unique")
print(f"{df_FOS.duplicated().sum()} : clé unique global")

0 : clé unique
0 : clé unique global


In [10]:
# Avec regex on va enelver le en:qsdfqsdf
# seulement si l'information fr existe (sinon on garde les en)
# ensuite on enlève en: et fr:
import re

for column in df.columns:
    p = re.compile(r'en:(.|\n)*?,')
    df_FOS[column] = [p.sub('', i) if 'fr:'in i else i for i in df_FOS[column]]
    p = re.compile(r'en:')
    df_FOS[column] = [p.sub('', x) for x in df_FOS[column]]
    p = re.compile(r'fr:')
    df_FOS[column] = [p.sub('', x) for x in df_FOS[column]]

In [11]:
# On nettoie vite fait
df_FOS = df_FOS.replace('\s+', ' ', regex=True)
df_FOS = df_FOS.replace(np.nan, '', regex=True)
df_FOS['additives_en'].head()

329       e14xx - modified starch, e451 - triphosphates,...
864865                                                     
864850                                                     
864832                                                     
864819    e410 - locust bean gum, e412 - guar gum, e428 ...
Name: additives_en, dtype: object

In [12]:
# Creation du nouvelle index propre
df_FOS["code"] = np.arange(1, len(df_FOS)+1)

In [13]:
df_FOS.head()

,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,code
329,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1514659309,2017-12-30t18:41:49z,1535310176,2018-08-26t19:02:56z,blanquette de volaille et son riz,,blanquette de volaille et son riz,,"carton, plastique","carton, plastique",,comme j’aime,comme-j-aime,"plats préparés, produits à la viande, plats pr...","meals, meat-based-products, meals-with-meat, p...","meals, meat-based products, meals with meat, p...",france,france,france,france,france,"viande française, fabriqué en france","viande-francaise, made-in-france","viande-francaise, made in france",,,,,,france,comme j'aime,france,france,france,"riz précuit 40, 4 % (eau, riz, huile de colza,...",,,,,,285 g,285,,2,,"e14xx, e451, e451i","e14xx - modified starch, e451 - triphosphates,...",0,,,0,,,0,b,4,composite foods,one-dish meals,"to-be-completed, nutrition-facts-completed, in...","to-be-completed, nutrit

In [14]:
# creation de colonne nb_vues

df_FOS['nb_vues'] = 0

In [17]:
df_FOS.head()

,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,code,nb_vues
329,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1514659309,2017-12-30t18:41:49z,1535310176,2018-08-26t19:02:56z,blanquette de volaille et son riz,,blanquette de volaille et son riz,,"carton, plastique","carton, plastique",,comme j’aime,comme-j-aime,"plats préparés, produits à la viande, plats pr...","meals, meat-based-products, meals-with-meat, p...","meals, meat-based products, meals with meat, p...",france,france,france,france,france,"viande française, fabriqué en france","viande-francaise, made-in-france","viande-francaise, made in france",,,,,,france,comme j'aime,france,france,france,"riz précuit 40, 4 % (eau, riz, huile de colza,...",,,,,,285 g,285,,2,,"e14xx, e451, e451i","e14xx - modified starch, e451 - triphosphates,...",0,,,0,,,0,b,4,composite foods,one-dish meals,"to-be-completed, nutrition-facts-completed, in...","to-be-completed

In [15]:
df_FOS.to_json (r'PRODUCTS_DB.json', index= True, orient= 'records')

In [16]:
df_FOS.to_csv('PRODUCTS_DB.csv')

In [32]:
for i in df_FOS.columns:
   if 'cat' in i: 
    print(i)

categories
categories_tags
categories_en
main_category
main_category_en


In [44]:
df_FOS['categories'].unique()
pd.DataFrame(df_FOS['categories'].unique()).to_csv("categories.csv")